In [7]:
import os

from autogen import ConversableAgent, UserProxyAgent, AssistantAgent
from dotenv import load_dotenv

load_dotenv()

True

In [38]:
llm_config={
    "config_list": [{
        "api_type": "mistral",
        "model": "mistral-large-latest",
        "api_key": os.environ.get("MISTRAL_API_KEY"),
        "temperature": 0.0,
    }]
}

In [3]:
agent = ConversableAgent(
    "chatbot",
    llm_config=llm_config,
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

In [4]:
reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "user"}])
print(reply)

{'content': "Why don't scientists trust atoms?\n\nBecause they make up everything!", 'role': 'assistant', 'function_call': None, 'tool_calls': None}


In [5]:
cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config=llm_config,
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config=llm_config,
    human_input_mode="NEVER",  # Never ask for human input.
)

In [6]:
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.", max_turns=2)

joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

Sure thing! Here's a joke for you:

Why did the tomato turn red?

Because it saw the salad dressing!

That's one of my favorites. I hope it brought a smile to your face! As a part of a comedian duo, I always try to have a few jokes up my sleeve to lighten the mood.

--------------------------------------------------------------------------------
joe (to cathy):

Joe, that was a hilarious joke! I really needed that laugh.

I'm glad I could make you laugh! Laughter is the best medicine, after all. And as a comedian, there's nothing more satisfying than making someone's day a little bit brighter with a good joke. If you ever need another laugh, just let me know!

--------------------------------------------------------------------------------
cathy (to joe):

Of course, I'm always here to make you laugh! And if you ever want to hear more about me and m

In [51]:
client_agent = ConversableAgent(
    "client",
    llm_config=False,
    human_input_mode="ALWAYS"
)

assistant_agent = ConversableAgent(
    "assistant",
    llm_config=llm_config,
    human_input_mode="NEVER",
    system_message="""
    You are Dona a swiss legal assistant, you need to help the client clarify their legal issue according to the swiss law.
    Summarize concisely the situation provided using appropriate legal terms.
    Start your summary by saying "Case Summary:".
    Be concise!
    Do not reference specific legal article numbers.
    Finish your reply by asking the client to either say "exit" if the summary is good or to provide any additional informations or corrections to edit the summary.
    If relevant and appropriate, suggest few examples of key informations that could be added to the summary.
    """
)

In [52]:
input_case = """
I have been accused of stealing a banana in a shop, I didn't take the receipt so I couldn't prove my innocence.
What am I risking?
"""

In [53]:
def my_summary_method(
    sender: ConversableAgent,
    recipient: ConversableAgent,
    summary_args: dict,
):
    last_msg_content = recipient.last_message(sender)["content"]
    splits = last_msg_content.split("\n")
    for split in splits:
        if split.lower().startswith("case summary:"):
            return split.replace("Case Summary:", "").strip()


chat_result = client_agent.initiate_chat(
    assistant_agent,
    message=input_case,
    summary_method=my_summary_method,
)

client (to assistant):


I have been accused of stealing a banana in a shop, I didn't take the receipt so I couldn't prove my innocence.
What am I risking?


--------------------------------------------------------------------------------
assistant (to client):

Case Summary: The client is facing an allegation of shoplifting, specifically the unlawful appropriation of a banana from a retail establishment. The client asserts their innocence but lacks a receipt to substantiate their claim. The client seeks to understand the potential legal consequences of this accusation under Swiss law.

Please provide any additional information or corrections to ensure the accuracy of this summary. If relevant, you may want to include details such as whether there were any witnesses, if you've been formally charged, or if you have any previous convictions. If the summary is satisfactory, please say "exit".

--------------------------------------------------------------------------------


In [50]:
chat_result.summary

'The client is facing an allegation of theft under Swiss law, specifically the unauthorized taking of a banana from a shop. The client maintains their innocence but is unable to provide a receipt as evidence. There were two witnesses present, but they have since left the scene.'